The notebook updates celestial coordinates (TARG_RA, TARG_DEC) at the midpoint of a JWST exposure (EXPMID), applying proper motion (MU_RA, MU_DEC) to celestial coordinates (PROP_RA, PROP_DEC, MU_EPOCH) provided by the observer. See Known Issues with JWST Data Products in JDox.

In [1]:
from math import cos, pi
import requests

from astropy.time import Time
from astropy.units import arcsec, deg, pc, yr
from astropy.io import fits

Define function that updates celestial coordinates (TARG_RA, TARG_DEC) at the midpoint of a JWST exposure (EXPMID), applying proper motion (MU_RA, MU_DEC) to celestial coordinates (PROP_RA, PROP_DEC, MU_EPOCH) provided by the observer.

In [2]:
def fix_radec(path, distance=1000*pc):
    '''Fix TARG_RA and TARG_DEC in primary FITS header.'''
    with fits.open(path, 'update') as hdulist:
        header = hdulist['primary'].header
        prop_ra = header['prop_ra'] # deg
        prop_dec = header['prop_dec'] # deg
        mu_epoch = Time(header['mu_epoch'])
        mu_ra = header['mu_ra'] # arcsec / yr
        mu_dec = header['mu_dec'] # arcsec / yr
        expmid = Time(header['expmid'], format='mjd')
        dtime = Time(expmid, format='mjd') - Time(mu_epoch)
        dyear = dtime.to(yr).value
        new_targ_ra = prop_ra + dyear * mu_ra / 3600 / cos(pi * prop_dec / 180)
        new_targ_dec = prop_dec + dyear * mu_dec / 3600
        
        old_targ_ra = header['targ_ra']
        old_targ_dec = header['targ_dec']
        print(f'old: targ_ra={old_targ_ra:.6f}, targ_dec={old_targ_dec:.6f}')
        header['targ_ra'] = new_targ_ra
        header['targ_dec'] = new_targ_dec
        print(f'new: targ_ra={new_targ_ra:.6f}, targ_dec={new_targ_dec:.6f}')

Define function to download a named file via the MAST API. The function includes authentication logic, but this example uses public data, so no MAST API token is required.

In [3]:
def get_jwst_file(name, mast_api_token=None):
    """Retrieve a JWST data file from MAST archive."""
    mast_url = "https://mast.stsci.edu/api/v0.1/Download/file"
    params = dict(uri=f"mast:JWST/product/{name}")
    if mast_api_token:
        headers = dict(Authorization=f"token {get_mast_api_token()}")
    else:
        headers = {}
    r = requests.get(mast_url, params=params, headers=headers, stream=True)
    r.raise_for_status()
    with open(name, "wb") as fobj:
        for chunk in r.iter_content(chunk_size=1024000):
            fobj.write(chunk)

Download a public, uncalibrated FITS file from the MAST archive.

In [4]:
name = 'jw02288003001_04102_00001_nrs1_cal.fits'
print(f'Downloading {name}')
get_jwst_file(name)

Update TARG_RA and TARG_DEC in the primary header.

In [5]:
fix_radec(name)

old: targ_ra=96.191172, targ_dec=-45.363249
new: targ_ra=96.191106, targ_dec=-45.362765
